0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [2]:
import torchvision
from torchvision import transforms
import torch
from PIL import Image

In [3]:
y = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

In [4]:
y = y.iloc[:,[1]]

In [5]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis')

Partial fit log reg to defy RAM Issues

In [6]:
from sklearn.linear_model import SGDClassifier

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()

In [34]:
help(Log)

Help on SGDClassifier in module sklearn.linear_model._stochastic_gradient object:

class SGDClassifier(BaseSGDClassifier)
 |  SGDClassifier(loss='hinge', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
 |  
 |  Linear classifiers (SVM, logistic regression, etc.) with SGD training.
 |  
 |  This estimator implements regularized linear models with stochastic
 |  gradient descent (SGD) learning: the gradient of the loss is estimated
 |  each sample at a time and the model is updated along the way with a
 |  decreasing strength schedule (aka learning rate). SGD allows minibatch
 |  (online/out-of-core) learning via the `partial_fit` method.
 |  For best results using the default learning rate schedule, the data sh

In [35]:
Log = SGDClassifier(loss='log_loss',penalty = 'l1')

Notes:
Log = SGDlassifier(loss='log_loss') : 50% training, 18% test accuracy.

In [43]:
results = {'SGDlassifier(loss=log_loss)':'50% training, 18% test',
          'SGDClassifier(loss=log_loss,penalty = l1': '40%training, 15% test'}

In [36]:
chunk = 1
i=0
j=3000
for df in os.listdir():
    print('currently processing chunk{chunk}/5'.format(chunk=chunk))
    X = pd.read_csv('X_{chunk}.csv'.format(chunk=chunk))
    print('Read CSV, now normalizing data...')
    X = scaler.fit_transform(X)
    print('Partial fitting the model...')
    Log.partial_fit(X,y[i:j], classes = np.unique(y))
    chunk=chunk+1
    if chunk == 5:
        X_pred_prob = Log.score(X,y[i:j])
        print('The accuracy for the training set is {prob}'.format(prob=X_pred_prob))
        break
    else:
        del X
        i=i+3000
        j=j+3000
        print('Will now continue with chunk {chunk}'.format(chunk=chunk))
   
    
    

currently processing chunk1/5
Read CSV, now normalizing data...
Partial fitting the model...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Will now continue with chunk 2
currently processing chunk2/5
Read CSV, now normalizing data...
Partial fitting the model...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Will now continue with chunk 3
currently processing chunk3/5
Read CSV, now normalizing data...
Partial fitting the model...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Will now continue with chunk 4
currently processing chunk4/5
Read CSV, now normalizing data...
Partial fitting the model...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy for the training set is 0.4033333333333333


In [27]:
chunk = 5

for df in os.listdir():
    print('currently processing chunk{chunk}/5'.format(chunk=chunk))
    X_test=pd.read_csv('X_{chunk}.csv'.format(chunk=chunk))
    if chunk == 5:
        break


currently processing chunk5/5


In [ ]:
%system free -m

In [ ]:
#X_test = pd.concat(X_test)

In [37]:
Log.score(scaler.transform(X_test),y[12000:14276])

0.15026362038664323